In [1]:
import segmentation_models_pytorch as smp

import torch

if __name__ == "__main__":
    import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet34",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
)
x = torch.randn(1, 3, 256, 256)

y = model(x)

print(y.shape)

torch.Size([1, 1, 256, 256])


In [2]:
from tqdm import tqdm
from torch import optim
import torch.utils.data
import torch.nn as nn
from detection import *
from utils import CellImageLoad
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import cv2

import argparse
import random
import os

#early stopping=1000にしている


def eval_net_img(
    net, dataset, save_dir,epoch,gpu=True, vis=None, vis_im=None, vis_gt=None, loss=nn.MSELoss()
):
    criterion = loss
    net.eval()
    losses = 0
    torch.cuda.empty_cache()
    
    if epoch % 100 == 0:
        for iteration, data in enumerate(dataset):
            img = data["image"]
            target = data["gt"]
            if gpu:
                img = img.cuda()
                target = target.cuda()

            pred_img = net(img)

           



            pred_img = pred_img.detach().cpu().numpy()
            pred_img=np.clip(pred_img,0,255)
            cv2.imwrite("{}/conf_epoch_{}_iter_{}.tif".format(save_dir,epoch,iteration), (img.detach().cpu().numpy() * 255).astype(np.uint8)[0].transpose(1, 2, 0))
            cv2.imwrite("{}/conf_gt_epoch_{}_iter_{}.tif".format(save_dir,epoch,iteration), (target.detach().cpu().numpy() * 255).astype(np.uint8)[0].transpose(1, 2, 0))
            cv2.imwrite("{}/conf_eval_epoch_{}_iter_{}.tif".format(save_dir,epoch,iteration), (pred_img * 255).astype(np.uint8)[0, 0])
            
    for iteration, data in enumerate(dataset):
        img = data["image"]
        target = data["gt"]
        if gpu:
            img = img.cuda()
            target = target.cuda()

        pred_img = net(img)

        loss = criterion(pred_img, target)
        losses += loss.data

    return losses / (iteration + 1)


def save_history(loss,val_loss, result_file):
        
    nb_epoch = len(loss)

    with open(result_file, "w") as fp:
        fp.write("epoch\tloss\tval_loss\n")
        for i in range(nb_epoch):
            fp.write("%d\t%f\t%f\n" % (i, loss[i], val_loss[i]))
def parse_args():
    """
    Parse input arguments
    """
    parser = argparse.ArgumentParser(description="Train data path")
    parser.add_argument(
        "-d",
        "--data_path",
        dest="data_path",
        help="training dataset's path",
        default="./data/σ=6/A=test/sarcoma_train",
        type=str,
    )
    parser.add_argument(
        "-v",
        "--val_path",
        dest="val_path",
        help="validation data path",
        default="./data/σ=6/A=test/sarcoma_val",
        type=str,
    )
    parser.add_argument(
        "-w",
        "--weight_path",
        dest="weight_path",
        help="save weight path",
        default="./data/σ=6/A=test/weight/nuclei_σ=6/best.pth",
    )
    parser.add_argument(
        "-g",
        "--gpu",
        dest="gpu",
        help="whether use CUDA",
        default=True,
        action="store_true",
    )
    parser.add_argument(
        "-b", "--batch_size", dest="batch_size", help="batch_size", default=3, type=int
    )
    parser.add_argument(
        "-e", "--epochs", dest="epochs", help="epochs", default=15001, type=int
    )
    parser.add_argument(
        "-l",
        "--learning_rate",
        dest="learning_rate",
        help="learning late",
        default=1e-3,
        type=float,
    )

    args = parser.parse_args()
    return args


class TrainNet:
    def __init__(self, args):
        data_loader = CellImageLoad(args.tra_img_paths, args.tra_gt_paths)
        self.train_dataset_loader = torch.utils.data.DataLoader(
            data_loader, batch_size=args.batch_size, shuffle=True, num_workers=0
        )
        self.number_of_traindata = data_loader.__len__()

        data_loader = CellImageLoad(args.val_img_paths, args.val_gt_paths)
        
        #validationのバッチサイズを1にしている
        self.val_loader = torch.utils.data.DataLoader(
            data_loader, batch_size=1, shuffle=False, num_workers=0
        )

        self.save_weight_path = args.weight_path
        
        self.save_img_path = args.img_path
        
        
        self.save_weight_path.parent.mkdir(parents=True, exist_ok=True)
        self.save_weight_path.parent.joinpath("epoch_weight").mkdir(
            parents=True, exist_ok=True
        )
        print(
            "Starting training:\nEpochs: {}\nBatch size: {} \nLearning rate: {}\ngpu:{}\n".format(
                args.epochs, args.batch_size, args.learning_rate, args.gpu
            )
        )

        self.net = net

        self.train = None
        self.val = None

        self.N_train = None
        self.optimizer = optim.Adam(net.parameters(), lr=args.learning_rate)
        self.epochs = args.epochs
        self.batch_size = args.batch_size
        self.gpu = args.gpu
        self.criterion = nn.MSELoss()
        self.losses = []
        self.val_losses = []
        self.evals = []
        self.epoch_loss = 0
        self.bad = 0

    def gather_path(self, train_paths, mode):
        ori_paths = []
        for train_path in train_paths:
            ori_paths.extend(sorted(train_path.joinpath(mode).glob("*.*")))
        return ori_paths

    def show_graph(self):
        x = list(range(len(self.losses)))
        plt.plot(x, self.losses)
        plt.plot(x, self.val_losses)
        plt.savefig(os.path.join(args.img_path, 'loss.png'))
        plt.show()
        plt.close()
        
        return self.losses,self.val_losses
        

    def loss_calculate(self, masks_probs_flat, true_masks_flat):
        return self.criterion(masks_probs_flat, true_masks_flat)

    def main(self):
        for epoch in range(self.epochs):
            net.train()
            print("Starting epoch {}/{}.".format(epoch + 1, self.epochs))

            pbar = tqdm(total=self.number_of_traindata)
            for i, data in enumerate(self.train_dataset_loader):
                imgs = data["image"]
                true_masks = data["gt"]

                if self.gpu:
                    imgs = imgs.cuda()
                    true_masks = true_masks.cuda()

                masks_pred = self.net(imgs)

                masks_probs_flat = masks_pred.view(-1)
                true_masks_flat = true_masks.view(-1)

                loss = self.loss_calculate(masks_probs_flat, true_masks_flat)
                self.epoch_loss += loss.item()

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                pbar.update(self.batch_size)
                
            #trainデータの確認用に追加
            log_dir='{}/train_img'.format(self.save_img_path)
            if not os.path.exists(log_dir):
                os.makedirs(log_dir)

            if epoch % 100 == 0:
                net.eval()
                pred_img = masks_pred.detach().cpu().numpy()
                pred_img=np.clip(pred_img,0,255)
                cv2.imwrite("{}/conf_epoch_{}_iter_{}.tif".format(log_dir,epoch,i), (imgs.detach().cpu().numpy() * 255).astype(np.uint8)[0].transpose(1, 2, 0))
                cv2.imwrite("{}/conf_gt_epoch_{}_iter_{}.tif".format(log_dir,epoch,i), (true_masks.detach().cpu().numpy() * 255).astype(np.uint8)[0].transpose(1, 2, 0))
                cv2.imwrite("{}/conf_eval_epoch_{}_iter_{}.tif".format(log_dir,epoch,i), (pred_img * 255).astype(np.uint8)[0, 0])
            pbar.close()
            self.validation(i, epoch)

            if self.bad >= 1000:
                print("stop running")
                break
#         self.show_graph()
        loss,val_loss=self.show_graph()
        return loss,val_loss

    def validation(self, number_of_train_data, epoch):
        loss = self.epoch_loss / (number_of_train_data + 1)
        print("Epoch finished ! Loss: {}".format(loss))

        self.losses.append(loss)
        if epoch % 100 == 0:
            torch.save(
                self.net.state_dict(),
                str(
                    self.save_weight_path.parent.joinpath(
                        "epoch_weight/{:05d}.pth".format(epoch)
                    )
                ),
            )
            
        #trainに使用しているデータの保存用
            
        save_dir='{}/val_img'.format(self.save_img_path)
        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        val_loss = eval_net_img(self.net, self.val_loader,save_dir,epoch,gpu=self.gpu)
        if loss < 0.1:
            print("val_loss: {}".format(val_loss))
            try:
                if min(self.val_losses) > val_loss:
                    print("update best")
                    torch.save(self.net.state_dict(), str(self.save_weight_path))
                    self.bad = 0
                else:
                    self.bad += 1
                    print("bad ++")
            except ValueError:
                torch.save(self.net.state_dict(), str(self.save_weight_path))
            self.val_losses.append(val_loss)
        else:
            print("loss is too large. Continue train")
            self.val_losses.append(val_loss)
        print("bad = {}".format(self.bad))
        self.epoch_loss = 0



        

In [1]:

from PIL import Image
import torch
import numpy as np
from pathlib import Path
import cv2

from utils import local_maxima, show_res, optimum, target_peaks_gen, remove_outside_plot
import argparse


def parse_args():
    """
    Parse input arguments
    """
    parser = argparse.ArgumentParser(description="Train data path")
    parser.add_argument(
        "-i",
        "--input_path",
        dest="input_path",
        help="dataset's path",
        default="./data/σ=6/A=test/sarcoma_test",
        type=str,
    )
    parser.add_argument(
        "-o",
        "--output_path",
        dest="output_path",
        help="output path",
        default="./output_sarcoma/A=test/σ=6/best_loss/peak_thresh=100_dist_peak=2_dist_thresh=30",
        type=str,
    )
    parser.add_argument(
        "-w",
        "--weight_path",
        dest="weight_path",
        help="load weight path",
        default="./data/σ=6/A=test/weight/nuclei_sarcoma/best.pth",
    )
    parser.add_argument(
        "-g", "--gpu", dest="gpu", help="whether use CUDA", action="store_true"
    )

    args = parser.parse_args()
    return args


class Predict:
    def __init__(self, args):
        self.net = args.net
        self.gpu = args.gpu

        self.ori_path = args.input_path / Path("ori")

        self.save_ori_path = args.output_path / Path("ori")
        self.save_pred_path = args.output_path / Path("pred")

        self.save_ori_path.mkdir(parents=True, exist_ok=True)
        self.save_pred_path.mkdir(parents=True, exist_ok=True)

    def pred(self, ori):
        img = ori.astype(np.float32) / ori.max()
        
        

        with torch.no_grad():
            
            
            img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0)
#            
           
            
            if self.gpu:
                img = img.cuda()
                
            
            mask_pred = self.net(img)
        
        
        pre_img = mask_pred.detach().cpu().numpy()[0, 0]
        #pre_img 0-
        pre_img=np.clip(pre_img,0,1.0)
        
        
        pre_img = (pre_img * 255).astype(np.uint8)
        
        return pre_img

    def main(self):
        self.net.eval()
        # path def
        paths = sorted(self.ori_path.glob("*.tif"))
        for i, path in enumerate(paths):
            
            ori=cv2.imread(str(path))
            

            ori = np.array(ori)
            
            pre_img = self.pred(ori)
            
            cv2.imwrite(str(self.save_pred_path / Path(f"{path.name}")), pre_img)
            cv2.imwrite(str(self.save_ori_path / Path(f"{path.name}")), ori)

def show_res(img, gt, res, no_detected_id, over_detection_id, associated_id, path=None):
    
    plt.figure(dpi=500)
    plt.figure()
    plt.imshow(img, plt.cm.gray)
    
    plt.plot(res[associated_id[:,1]][:, 0], res[associated_id[:,1]][:, 1],'g', marker='o',label="True positive", markerfacecolor="None", markersize=10,linestyle='None')
    
    
   
    
  
    if no_detected_id.shape[0] > 0:
        plt.plot(
            gt[no_detected_id][:, 0], gt[no_detected_id][:, 1], "b2", marker='o',label="False negative", markerfacecolor="None", markersize=10)
    if over_detection_id.shape[0] > 0:
        plt.plot(res[over_detection_id][:, 0], res[over_detection_id][:, 1], "k1", marker='o',label="False positive", markerfacecolor="None", markersize=10)
    plt.legend(bbox_to_anchor=(0, 1.05), loc="upper left", fontsize=4, ncol=4)
    plt.axis("off")
    
    plt.savefig(path, bbox_inches='tight', pad_inches=0, trasparent=True,dpi=500)
    plt.close()
    
    

class PredictFmeasure(Predict):
    def __init__(self, args):
        super().__init__(args)
        self.ori_path = args.input_path / Path("ori")
        self.gt_path = args.input_path / Path("likeli")

        self.save_gt_path = args.output_path / Path("gt")
        self.save_error_path = args.output_path / Path("error")
        self.save_txt_path = args.output_path / Path("f-measure.txt")

        self.save_gt_path.mkdir(parents=True, exist_ok=True)
        self.save_error_path.mkdir(parents=True, exist_ok=True)
        

        self.peak_thresh = args.peak_thresh

        self.dist_peak =args.dist_peak
        
     
        self.dist_threshold = args.dist_thresh

        self.tps = 0
        self.fps = 0
        self.fns = 0

    def cal_tp_fp_fn(self, ori, gt_img, pre_img, i, path):
        

        thresh,bin_img = cv2.threshold(pre_img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        mask_img=255-bin_img
        kernel = np.ones((3,3),np.uint8)
        mask_img = cv2.morphologyEx(mask_img,cv2.MORPH_OPEN,kernel,iterations = 2)
        
        pre_img=cv2.bitwise_and(pre_img, mask_img)
        
        gt = target_peaks_gen((gt_img).astype(np.uint8))
        res = local_maxima(pre_img, self.peak_thresh, self.dist_peak)
        
        
        
        associate_id = optimum(gt, res, self.dist_threshold)
        associate_id = np.array(associate_id,dtype='int')
        
        
        

        gt_final, no_detected_id = remove_outside_plot(
            gt, associate_id, 0, pre_img.shape
        )
        res_final, overdetection_id = remove_outside_plot(
            res, associate_id, 1, pre_img.shape
        )
        

        show_res(
            ori,
            gt,
            res,
            no_detected_id,
            overdetection_id,
            associate_id,
            path=str(self.save_error_path / Path(f'{path.name}')),
        )
    
        cv2.imwrite(str(self.save_pred_path / Path(f'{path.name}')), pre_img)
        cv2.imwrite(str(self.save_ori_path / Path(f'{path.name}')), ori)
        cv2.imwrite(str(self.save_gt_path / Path(f'{path.name}')), gt_img)

        tp = associate_id.shape[0]
        fn = gt_final.shape[0] - associate_id.shape[0]
        fp = res_final.shape[0] - associate_id.shape[0]
        self.tps += tp
        self.fns += fn
        self.fps += fp

    def main(self):
        self.net.eval()
        # path def
        path_x = sorted(self.ori_path.glob("*.*"))
        path_y = sorted(self.gt_path.glob("*.*"))

        z = zip(path_x, path_y)

        for i, b in enumerate(z):
            import gc

            gc.collect()
            ori = cv2.imread(str(b[0]))

            gt_img = cv2.imread(str(b[1]), 0)

            

            pre_img = self.pred(ori)

            self.cal_tp_fp_fn(ori, gt_img, pre_img, i,b[0])
        if self.tps == 0:
            f_measure = 0
            recall=0
            precision=0
        else:
            recall = self.tps / (self.tps + self.fns)
            precision = self.tps / (self.tps + self.fps)
            f_measure = (2 * recall * precision) / (recall + precision)

        print(precision, recall, f_measure)
        with self.save_txt_path.open(mode="a") as f:
            f.write("%f,%f,%f\n" % (precision, recall, f_measure))




evaluate the model's performance

In [77]:

group_list=['A','B','C','D','E']



loss_type_list=['best','last','best_F']





data_type_list=['test','train','val']





peak_thresh=100

dist_peak=20

dist_thresh=20



for loss_type in loss_type_list:
        print(loss_type)

        for data_type in data_type_list:
                print(data_type)
                for group in group_list:
                        print(group)
                        


                        parser = argparse.ArgumentParser()

                        parser.add_argument('--input_path',help="dataset's path",
                                default=f'./data_add/σ=6/{group}=test/sarcoma_test',
                                type=str)
                        parser.add_argument("--output_path",
                                dest="output_path",
                                help="output path",
                                default=f"./output_add/σ=6/{group}=test/{loss_type}_loss/peak_thresh={peak_thresh}_dist_peak={dist_peak}_dist_thresh={dist_thresh}",
                                type=str,)

                        if loss_type=='best':

                                parser.add_argument( "-w",
                                        "--weight_path",
                                        dest="weight_path",
                                        help="load weight path",
                                        default=f"./data_add/σ=6/{group}=test/weight_F/nuclei_sarcoma/best.pth",)
                                
                                
                        elif loss_type=='best_F':

                                parser.add_argument( "-w",
                                        "--weight_path",
                                        dest="weight_path",
                                        help="load weight path",
                                        default=f"./data_add/σ=6/{group}=test/weight_F/nuclei_sarcoma/best_F.pth",)
                        else:

                                parser.add_argument( "-w",
                                        "--weight_path",
                                        dest="weight_path",
                                        help="load weight path",
                                        default=f"./data_add/σ=6/{group}=test/weight_F/nuclei_sarcoma/epoch_weight/01000.pth",)

                        parser.add_argument('--peak_thresh',help="peak_thresh",
                                default=f'{peak_thresh}', type=int)
                        parser.add_argument('--dist_thresh',help="dist_thresh",
                                default=f'{dist_thresh}', type=int)

                        parser.add_argument('--dist_peak',help="dist_peak",
                        default=f'{dist_peak}', type=int)
                        

                        parser.add_argument("-g", "--gpu", dest="gpu", help="whether use CUDA", action="store_true")





                        if __name__ == "__main__":
                                args = parser.parse_args(args=["--input_path",f"./data/σ=6/{group}=test/sarcoma_{data_type}",  "--output_path", f"./output/σ=6/{group}=test/{data_type}/{loss_type}/peak_thresh={peak_thresh}_dist_peak={dist_peak}_dist_thresh={dist_thresh}",])

                                args.input_path = Path(args.input_path)
                                args.output_path = Path(args.output_path)

                                net = model
                                net.load_state_dict(torch.load(args.weight_path, map_location="cpu"))

                                if args.gpu:
                                        net.cuda()
                                args.net = net


                                # # prediction だけ
                                pred = Predict(args)
                                # 
                                pred.main()
                                
                                pred_F=PredictFmeasure(args)
                                pred_F.main()
    
    
    

best
test
A
0.7193251533742331 0.678726483357453 0.6984363365599403
B
0.8623188405797102 0.45857418111753373 0.59874213836478
C
0.7727272727272727 0.6993670886075949 0.7342192691029901
D


/home/kazuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:116: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


0.6666666666666666 0.7587719298245614 0.7097435897435896
E


/home/kazuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/home/kazuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:116: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/home/kazuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see

0.6924493554327809 0.7657841140529531 0.7272727272727273
train
A


...

<Figure size 3000x2000 with 0 Axes>

<Figure size 3000x2000 with 0 Axes>

<Figure size 3000x2000 with 0 Axes>

<Figure size 3000x2000 with 0 Axes>

<Figure size 3000x2000 with 0 Axes>